Templates for Python Programmers
====

[The Django template language: for Python programmers | Django documentation | Django ](https://docs.djangoproject.com/en/2.0/ref/templates/api/)

本文档从技术角度解释了Django模板系统 - 它是如何工作的以及如何扩展它。如果您只是在查找有关语言语法的参考，请参阅Django模板语言。

# Overview
在Python中使用模板系统有三个步骤：

1. 配置一个引擎。
2. 将模板代码编译到模板中。
3. 使用 Context 渲染模板。

Django项目通常依赖于这些步骤中的高级别后端不可知API，而不是模板系统的低级 API：

1. 对于TEMPLATES设置中的每个DjangoTemplates后端，Django实例化一个引擎。 DjangoTemplates包装引擎并将其调整为通用模板后端API。

2. django.template.loader模块提供了用于加载模板的函数，例如get_template（）。他们返回一个包装实际的django.template.Template的django.template.backends.django.Template。

3. 在前一步中获得的模板有一个render（）方法，该方法将上下文和可能的请求封送到Context中，并将渲染委托给底层的Template。

# 配置引擎

如果你只是使用`DjangoTemplates`后端，这可能不是你正在寻找的文档。下面描述的Engine类的一个实例可以使用该后端的引擎属性访问，并且下面提到的任何属性默认值都被`DjangoTemplates`传递的内容覆盖。

```
class Engine(dirs=None, app_dirs=False, context_processors=None, debug=False, 
loaders=None, string_if_invalid='', file_charset='utf-8', libraries=None, 
builtins=None, autoescape=True)[source]

```

在实例化Engine时，所有参数都必须作为关键字参数传递：

**dirs** 是引擎应该查找模板源文件的目录列表。它用于配置文件系统.

**Loader** 它默认为一个空列表。

**app_dirs** 只影响装载器的默认值。见下文。它默认为False。

**autoescape** 控制是否启用HTML自动转义。它默认为True。

**警告** 如果您要呈现非HTML模板，请将其设置为False！

`context_processors` 是用于可调参数的点状Python路径的列表，用于在使用请求呈现模板时填充上下文。这些可调用对象将请求对象作为参数，并返回要合并到上下文中的项的字典。

它默认为一个空列表。有关更多信息，请参阅RequestContext。

**debug** 是一个打开/关闭模板调试模式的布尔值。如果它为True，则模板引擎将存储其他调试信息，这些信息可用于显示模板呈现期间引发的任何异常的详细报告。

它默认为False。

**Loaders**，加载器，是模板加载器类的列表，指定为字符串。每个`Loader`类都知道如何从特定源文件导入模板。 或者，可以使用元组而不是字符串。元组中的第一项应该是Loader类名，后续项在初始化期间传递给Loader。

它默认为一个包含以下内容的列表：
- 'django.template.loaders.filesystem.Loader'
- 'django.template.loaders.app_directories.Loader'当且仅当app_dirs为True。

如果debug为False，则这些加载器将被包装在django.template.loaders.cached.Loader中。

加载器的细节请参见：[Loader types](https://docs.djangoproject.com/en/2.0/ref/templates/api/#template-loaders)

**string_if_invalid** 是模板系统应用于无效（例如拼写错误）变量的输出（以字符串形式）。它默认为空字符串。请参阅处理无效变量以了解详细信息。

**file_charset** 是用于读取磁盘上模板文件的字符集。它默认为'utf-8'。

**Libraries**：模板标签模块的标签字典和点状Python路径，以向模板引擎注册。这用于添加新库或为现有库提供替代标签。例如：

```
Engine(
    libraries={
        'myapp_tags': 'path.to.myapp.tags',
        'admin.urls': 'django.contrib.admin.templatetags.admin_urls',
    },
)
```

可以通过将相应的字典键传递给{％load％}标签来加载库。

**'builtins'**: 
要添加到内置插件的模板标记模块的点Python路径列表。例如：

```
Engine(
    builtins=['myapp.builtins'],
)
```

无需先调用{％load％}标记，即可使用内置库中的标记和过滤器。

静态 ** `Engine.get_default()[source]`**

从第一个配置的DjangoTemplates引擎返回底层引擎。如果没有引擎配置，引发ImproperlyConfigured错误。

它需要保留依赖全局可用，隐式配置引擎的API。任何其他用途强烈不鼓励。

**`Engine.from_string(template_code)[source]`**

编译给出的模板代码，并返回一个模板对象。

**`Engine.get_template(template_name)[source]`**

加载给出的名字的模板，编译它并返回一个模板对象。

**`Engine.select_template(template_name_list)[source]`**

像`get_template（）`一样，除了它需要一个名称列表并返回找到的第一个模板。

## Loading a template

推荐的创建模板的方法是调用引擎的工厂方法：get_template（），select_template（）和from_string（）。

在一个Django项目中，`TEMPLATES`设置定义了一个`DjangoTemplates`引擎，可以直接实例化一个模板。如果定义了多个`DjangoTemplates`引擎，则会使用第一个引擎。

**`class Template[source]`**

这个类位于django.template.Template。构造函数接受一个参数 - 原始模板代码：

```
from django.template import Template

template = Template("My name is {{ my_name }}.")
```

**幕后**

系统只会解析您的原始模板代码 - 在您创建模板对象时。从那时起，它作为一个树形结构存储在内部以提高性能。

即使解析本身也很快。 大多数解析通过对单个简短正则表达式的单个调用发生。

## Rendering a context

一旦你有一个编译的模板对象，你可以用它来渲染一个Context。您可以重复使用相同的模板以不同的Context多次渲染它。

**`class Context(dict_=None)[source]`**

django.template.Context的构造函数接受一个可选参数 - 一个将变量名称映射到变量值的字典。

有关详情，请参阅下面的 [Playing with Context objects](https://docs.djangoproject.com/en/2.0/ref/templates/api/#playing-with-context)。

**`Template.render(context)[source]`**

使用Context调用Template对象的render（）方法来“填充”模板：

```
>>> from django.template import Context, Template
>>> template = Template("My name is {{ my_name }}.")

>>> context = Context({"my_name": "Adrian"})
>>> template.render(context)
"My name is Adrian."

>>> context = Context({"my_name": "Dolores"})
>>> template.render(context)
"My name is Dolores."
```

## 变量和查找
变量名称必须是包含任意字母（A-Z），任意数字（0-9），下划线（但不得以下划线开头）或点的字符组成。

点在模板渲染中有特殊的含义。变量名称中的点表示查找。特别是，当模板系统遇到变量名称中的点时，它会按以下顺序尝试以下查找：

- 字典查找。例如：foo [“bar”]
- 属性查找。例如：foo.bar
- 列表索引查找。例如：foo [bar]

请注意，模板表达式{{foo.bar}}中的“bar”将被解释为文字字符串，并且不会使用变量“bar”的值（如果存在于模板Context中）。

模板系统使用可用的第一个查找类型。 这是短路逻辑。 这里有一些例子：

```
>>> from django.template import Context, Template
>>> t = Template("My name is {{ person.first_name }}.")
>>> d = {"person": {"first_name": "Joe", "last_name": "Johnson"}}
>>> t.render(Context(d))
"My name is Joe."

>>> class PersonClass: pass
>>> p = PersonClass()
>>> p.first_name = "Ron"
>>> p.last_name = "Nasty"
>>> t.render(Context({"person": p}))
"My name is Ron."

>>> t = Template("The first stooge in the list is {{ stooges.0 }}.")
>>> c = Context({"stooges": ["Larry", "Curly", "Moe"]})
>>> t.render(c)
"The first stooge in the list is Larry."
```

如果变量的某个部分是可调用的，模板系统将尝试调用它：

```
>>> class PersonClass2:
...     def name(self):
...         return "Samantha"
>>> t = Template("My name is {{ person.name }}.")
>>> t.render(Context({"person": PersonClass2}))
"My name is Samantha."

```

可调用变量比仅需要直接查找的变量稍微复杂一些。这里有一些事情要记住：

> 如果变量在调用时引发异常，则将传播异常，除非异常的属性silent_variable_failure的值为True。如果异常确实具有值为True的silent_variable_failure属性，则该变量将呈现为引擎的string_if_invalid配置选项的值（默认情况下为空字符串）。 例：

```
>>> t = Template("My name is {{ person.first_name }}.")
>>> class PersonClass3:
...     def first_name(self):
...         raise AssertionError("foo")
>>> p = PersonClass3()
>>> t.render(Context({"person": p}))
Traceback (most recent call last):
...
AssertionError: foo

>>> class SilentAssertionError(Exception):
...     silent_variable_failure = True
>>> class PersonClass4:
...     def first_name(self):
...         raise SilentAssertionError
>>> p = PersonClass4()
>>> t.render(Context({"person": p}))
"My name is ."
```

请注意，django.core.exceptions.ObjectDoesNotExist（它是所有Django数据库API DoesNotExist异常的基类）具有silent_variable_failure = True。 因此，如果您将Django模板与Django模型对象一起使用，则任何DoesNotExist异常都将自动失败。

只有在没有必要参数的情况下才能调用变量。否则，系统将返回引擎的string_if_invalid选项的值。

显然，在调用某些变量时可能会有副作用，它可能是愚蠢的或安全漏洞，以允许模板系统访问它们。

一个很好的例子是每个Django模型对象上的delete（）方法。模板系统不应该被允许做这样的事情：

`I will now delete this valuable data. {{ data.delete }}`

为了防止这种情况，请在可调用变量上设置alters_data属性。如果模板系统具有alters_data = True设置，则模板系统将不会调用变量，而是无条件地将该变量替换为string_if_invalid。 Django模型对象上的动态生成的delete（）和save（）方法自动获取alters_data = True。例：
```
def sensitive_function(self):
    self.database_record.delete()
sensitive_function.alters_data = True
```

有时您可能会因为其他原因关闭此功能，并告诉模板系统不管什么都留下未调用的变量。为此，请在值为True的可调用对象上设置do_not_call_in_templates属性。然后，模板系统将像您的变量不可调用一样工作（例如，允许您访问可调用的属性）。

# 如何处理变量

通常，如果变量不存在，模板系统会插入引擎的`string_if_invalid`配置选项的值，该选项默认设置为''（空字符串）。

只有在string_if_invalid设置为''（空字符串）时，才会应用应用于无效变量的过滤器。如果string_if_invalid被设置为任何其他值，则变量过滤器将被忽略。

这种行为与if，for和regroup模板标签略有不同。 如果向这些模板标签中的一个提供了无效变量，则该变量将被解释为无。 过滤器始终应用于这些模板标记中的无效变量。

如果string_if_invalid包含'％s'，则格式标记将替换为无效变量的名称。

**仅用于调试目的！**

虽然string_if_invalid可以成为一个有用的调试工具，但将其作为“开发默认”打开它并不是个好主意。

当遇到不存在的变量时，许多模板（包括管理站点中的模板）都依赖模板系统的静默。 如果您将“'以外的值分配给string_if_invalid，那么您将遇到这些模板和网站的渲染问题。

一般来说，只有为了调试特定的模板问题才启用string_if_invalid，然后在调试完成后清除。

## 内置变量

每个上下文都包含True，False和None。 如您所料，这些变量将解析为相应的Python对象。

# 字符串文字的限制

Django的模板语言无法转义用于自己语法的字符。 例如，如果需要输出像{％和％}这样的字符序列，则templatetag标签是必需的。

如果要将这些序列包含在模板过滤器或标记参数中，则存在类似的问题。 例如，在解析块标记时，Django的模板解析器在{％之后查找％}的第一个匹配项。 这可以防止使用“％}”作为字符串文字。 例如，将为以下表达式引发TemplateSyntaxError：

```
{% include "template.html" tvar="Some string literal with %} in it." %}

{% with tvar="Some string literal with %} in it." %}{% endwith %}
```

在过滤器参数中使用保留序列可触发相同的问题：

`{{ some.variable|default:"}}" }}`

如果您需要在这些序列中使用字符串，请将它们存储在模板变量中，或使用自定义模板标记或过滤器来解决该限制。

## 玩转Context对象

大多数情况下，您将通过将完全填充的字典传递给Context（）来实例化Context对象。但是，使用标准字典语法，也可以在实例化`Context`对象后添加和删除项目：

```
>>> from django.template import Context
>>> c = Context({"foo": "bar"})
>>> c['foo']
'bar'
>>> del c['foo']
>>> c['foo']
Traceback (most recent call last):
...
KeyError: 'foo'
>>> c['newvariable'] = 'hello'
>>> c['newvariable']
'hello'
```

**`Context.get(key, otherwise=None)`**

如果key在Context中，则返回键的值，否则返回otherwise。

**`Context.setdefault(key, default=None)`**
如果键在Context中，则返回其值。 否则，插入具有默认值的key并返回默认值。

**`Context.pop()`**

**`Context.push()`**

**`exception ContextPopException[source]`**

一个Context对象是一个堆栈。也就是说，你可以push（）和pop（）它。如果你弹出（）太多，它会引发django.template.ContextPopException：

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> c.push()
{}
>>> c['foo'] = 'second level'
>>> c['foo']
'second level'
>>> c.pop()
{'foo': 'second level'}
>>> c['foo']
'first level'
>>> c['foo'] = 'overwritten'
>>> c['foo']
'overwritten'
>>> c.pop()
Traceback (most recent call last):
...
ContextPopException
```


您还可以使用push（）作为 Context 管理器来确保调用匹配的pop（）。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> with c.push():
...     c['foo'] = 'second level'
...     c['foo']
'second level'
>>> c['foo']
'first level'

```

所有传递给push（）的参数都会传递给dict构造函数，用于构建新的Context级别。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> with c.push(foo='second level'):
...     c['foo']
'second level'
>>> c['foo']
'first level'
```

**`Context.update(other_dict)[source]`**

除了push（）和pop（）之外，Context对象还定义了一个update（）方法。 这与push（）一样工作，但将字典作为参数并将该字典推送到堆栈而不是空字符。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> c.update({'foo': 'updated'})
{'foo': 'updated'}
>>> c['foo']
'updated'
>>> c.pop()
{'foo': 'updated'}
>>> c['foo']
'first level'
```

与push（）类似，您可以使用update（）作为上下文管理器来确保调用匹配的pop（）。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> with c.update({'foo': 'second level'}):
...     c['foo']
'second level'
>>> c['foo']
'first level'
```

使用 Context 作为堆栈在一些自定义模板标签中迟早会派上用场，[custom template tags.](https://docs.djangoproject.com/en/2.0/howto/custom-template-tags/#howto-writing-custom-template-tags)

**`Context.flatten()`**

使用flatten（）方法，您可以将整个Context堆栈作为一个字典包括内置变量。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> c.update({'bar': 'second level'})
{'bar': 'second level'}
>>> c.flatten()
{'True': True, 'None': None, 'foo': 'first level', 'False': False, 
'bar': 'second level'}
```
flatten（）方法也在内部用于使Context对象具有可比性。

```
>>> c1 = Context()
>>> c1['foo'] = 'first level'
>>> c1['bar'] = 'second level'
>>> c2 = Context()
>>> c2.update({'bar': 'second level', 'foo': 'first level'})
{'foo': 'first level', 'bar': 'second level'}
>>> c1 == c2
True
```

flatten（）的结果在单元测试中可以用于比较Context和dict：

```
class ContextTest(unittest.TestCase):
    def test_against_dictionary(self):
        c1 = Context()
        c1['update'] = 'value'
        self.assertEqual(c1.flatten(), {
            'True': True,
            'None': None,
            'False': False,
            'update': 'value',
        })
 ```

## 使用RequestContext

**```
class RequestContext(request, dict_=None, processors=None)[source]
```**

Django带有一个特殊的Context类django.template.RequestContext，它与普通的django.template.Context稍有不同。 第一个区别是它将HttpRequest作为第一个参数。 例如：

```
c = RequestContext(request, {
    'foo': 'bar',
})

```

第二个区别是，根据引擎的context_processors配置选项，它会自动使用几个变量填充Context。

context_processors选项是一个可调用的列表 - 称为上下文处理器 - 将请求对象作为参数，并返回要合并到上下文中的项目字典。 在默认生成的设置文件中，默认模板引擎包含以下上下文处理器：

```
[
    'django.template.context_processors.debug',
    'django.template.context_processors.request',
    'django.contrib.auth.context_processors.auth',
    'django.contrib.messages.context_processors.messages',
]
```

除此之外，RequestContext总是启用'django.template.context_processors.csrf'。 这是管理员和其他contrib应用程序所需的与安全相关的上下文处理器，并且在意外错误配置的情况下，它将被故意硬编码并且无法在context_processors选项中关闭。

每个处理器按顺序应用。这意味着，如果一个处理器向上下文添加变量并且第二个处理器添加一个具有相同名称的变量，则第二个处理器将覆盖第一个处理器。 下面将解释默认的处理器。

**当应用上下文处理器时**

上下文处理器应用于上下文数据之上，即上下文处理器优先于上下文数据。这意味着上下文处理器可能会覆盖您提供给Context或RequestContext的变量，因此请注意避免变量名与您的上下文处理器提供的变量名重叠。

如果您希望上下文数据优先于上下文处理器，请使用以下模式：

```
from django.template import RequestContext

request_context = RequestContext(request)
request_context.push({"my_name": "Adrian"})
```
Django这样做是为了允许上下文数据覆盖API（如render（）和TemplateResponse）中的上下文处理器。

此外，您可以使用可选的第三个参数处理器为RequestContext提供附加处理器的列表。 在这个例子中，RequestContext实例获取一个ip_address变量：

```
from django.http import HttpResponse
from django.template import RequestContext, Template

def ip_address_processor(request):
    return {'ip_address': request.META['REMOTE_ADDR']}

def client_ip_view(request):
    template = Template('{{ title }}: {{ ip_address }}')
    context = RequestContext(request, {
        'title': 'Your IP Address',
    }, [ip_address_processor])
    return HttpResponse(template.render(context))
```

# 内置模板上下文处理器
以下是每个内置处理器的功能：

**`django.contrib.auth.context_processors.auth`**

**auth()[source]**

如果启用此处理器，则每个RequestContext都将包含这些变量：

- user - 代表当前登录用户的auth.User实例（或者AnonymousUser实例，如果客户端未登录）。
- perms - django.contrib.auth.context_processors.PermWrapper的一个实例，表示当前登录的用户拥有的权限。

**`django.template.context_processors.debug`**

**debug()[source]**

如果启用此处理器，则每个RequestContext都将包含这两个变量 - 但仅当您的DEBUG设置设置为True且请求的IP地址（request.META ['REMOTE_ADDR']）处于INTERNAL_IPS设置时：

- 调试 - True。 您可以在模板中使用它来测试您是否处于DEBUG模式。
- sql_queries - {'sql'：...，'time'：...}字典的列表，表示在请求期间发生的每个SQL查询以及需要多长时间。 该列表按数据库别名顺序排列，然后按查询顺序排列。 它在访问中延迟生成。

**`django.template.context_processors.i18n`**
如果启用此处理器，则每个RequestContext都将包含这两个变量：

- LANGUAGE - LANGUAGES设置的值。
- LANGUAGE_CODE - request.LANGUAGE_CODE，如果存在。 否则，LANGUAGE_CODE设置的值。
请参阅更多国际化和本地化。

**`django.template.context_processors.media`**

如果启用此处理器，则每个RequestContext都将包含一个变量MEDIA_URL，提供MEDIA_URL设置的值。

**`django.template.context_processors.static`**

**static()[source]**

如果启用此处理器，则每个RequestContext都将包含一个变量MEDIA_URL，提供MEDIA_URL设置的值。

**`django.template.context_processors.csrf`**

此处理器会添加csrf_token模板标记所需的令牌，以防止跨站请求伪造。

**`django.template.context_processors.request`**

如果启用此处理器，则每个RequestContext都将包含一个变量请求，即当前的HttpRequest。

**`django.template.context_processors.tz`**

**tz()[source]**

如果启用此处理器，则每个RequestContext都将包含一个变量TIME_ZONE，提供当前活动时区的名称。

**`django.contrib.messages.context_processors.`**

如果启用此处理器，则每个RequestContext都将包含这两个变量：

- messages - 通过消息框架设置的消息列表（作为字符串）。
- DEFAULT_MESSAGE_LEVELS - 消息级别名称与其数值的映射。

# 加载模板

通常，您将模板存储在文件系统的文件中，而不是自己使用低级别的模板API。 将模板保存在指定为模板目录的目录中。

Django在很多地方搜索模板目录，具体取决于您的模板加载设置（请参阅下面的“加载器类型”），但指定模板目录的最基本方法是使用DIRS选项。

## DIRS选项
通过使用设置文件中TEMPLATES设置中的DIRS选项或者Engine的dirs参数来告诉Django你的模板目录是什么。 这应该设置为包含模板目录完整路径的字符串列表：

```
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [
            '/home/html/templates/lawrence.com',
            '/home/html/templates/default',
        ],
    },
]
```

只要目录和模板可以被Web服务器读取，你的模板就可以到达任何你想要的地方。 他们可以有任何你想要的扩展名，例如.html或.txt，或者他们可以没有扩展名。

请注意，即使在Windows上，这些路径也应该使用Unix风格的正斜杠。

## 加载器类型

默认情况下，Django使用基于文件系统的模板加载器，但Django附带了一些其他模板加载器，它们知道如何从其他源加载模板。

这些其他装载器中的一些默认情况下是禁用的，但您可以通过在“模板”设置中将“装载器”选项添加到DjangoTemplates后端或将装载器参数传递给引擎来激活它们。 加载器应该是一个字符串或元组列表，其中每个表示一个模板加载器类。 以下是Django附带的模板加载器：

**·django.template.loaders.filesystem.Loader**

**class filesystem.Loader**

根据DIRS，从文件系统加载模板。

该加载器默认启用。但是，除非将DIRS设置为非空列表，否则它不会找到任何模板：

```
TEMPLATES = [{
    'BACKEND': 'django.template.backends.django.DjangoTemplates',
    'DIRS': [os.path.join(BASE_DIR, 'templates')],
}]
```

您也可以覆盖'DIRS'并为特定的文件系统加载器指定特定的目录：

```
TEMPLATES = [{
    'BACKEND': 'django.template.backends.django.DjangoTemplates',
    'OPTIONS': {
        'loaders': [
            (
                'django.template.loaders.filesystem.Loader',
                [os.path.join(BASE_DIR, 'templates')],
            ),
        ],
    },
}]
```

**`django.template.loaders.app_directories.Loader`**

**class app_directories.Loader**

从文件系统中的Django应用程序加载模板。 对于INSTALLED_APPS中的每个应用程序，加载器查找模板子目录。 如果该目录存在，Django将在其中查找模板。

这意味着您可以将模板与个人应用程序一起存储。 这也使得使用默认模板分发Django应用程序变得很容易。

例如，对于此设置：

`INSTALLED_APPS = ['myproject.polls', 'myproject.music']`

...然后get_template（'foo.html'）将按以下顺序在这些目录中查找foo.html：

- /path/to/myproject/polls/templates/
- /path/to/myproject/music/templates/

...并将使用它找到的第一个。

INSTALLED_APPS的顺序非常重要！ 例如，如果您想自定义Django管理员，您可以选择使用myproject.polls中的admin / base_site.html从django.contrib.admin覆盖标准admin / base_site.html模板。 您必须确保myproject.polls在INSTALLED_APPS中的django.contrib.admin之前，否则django.contrib.admin将首先加载，您的将被忽略。

请注意，加载器在第一次运行时执行优化：它缓存一个INSTALLED_APPS包具有模板子目录的列表。

只需将APP_DIRS设置为True即可启用该加载器：

```
TEMPLATES = [{
    'BACKEND': 'django.template.backends.django.DjangoTemplates',
    'APP_DIRS': True,
}]
```

**`django.template.loaders.cached.Loader`**

**class cached.Loader**

默认情况下（当DEBUG为True时），模板系统每次渲染时都会读取和编译模板。 虽然Django模板系统速度很快，但阅读和编译模板的开销可能会增加。

您可以使用它应该包装的其他加载程序的列表配置缓存的模板加载程序。 当第一次遇到未知模板时，使用包装的加载器来定位未知模板。 缓存的加载器然后将编译后的模板存储在内存中。 缓存的Template实例返回后续请求加载相同的模板。

如果未指定OPTIONS ['loaders']并且OPTIONS ['debug']为False（后者选项默认为DEBUG的值），则自动启用此加载器。

您也可以使用像这样的设置启用模板缓存与一些自定义模板加载器：

```
TEMPLATES = [{
    'BACKEND': 'django.template.backends.django.DjangoTemplates',
    'DIRS': [os.path.join(BASE_DIR, 'templates')],
    'OPTIONS': {
        'loaders': [
            ('django.template.loaders.cached.Loader', [
                'django.template.loaders.filesystem.Loader',
                'django.template.loaders.app_directories.Loader',
                'path.to.custom.Loader',
            ]),
        ],
    },
}]
```

**`django.template.loaders.locmem.Loader`**

**class locmem.Loader**

从Python字典加载模板。 这对于测试很有用。

该加载器将模板字典作为其第一个参数：

```
TEMPLATES = [{
    'BACKEND': 'django.template.backends.django.DjangoTemplates',
    'OPTIONS': {
        'loaders': [
            ('django.template.loaders.locmem.Loader', {
                'index.html': 'content here',
            }),
        ],
    },
}]
```

该加载器默认是禁用的。

Django根据“加载器”选项依次使用模板加载器。 它使用每个加载器，直到加载器找到匹配。

## 加载器方法

**class Loader[source]**

加载来自给定源的模板，例如文件系统或数据库。

**`get_template_sources(template_name)[source]`**

采用template_name并为每个可能的源生成Origin实例的方法。

例如，文件系统加载器可能会接收'index.html'作为template_name参数。 这个方法会产生index.html完整路径的原点，因为它出现在加载器看到的每个模板目录中。

该方法不需要验证模板是否存在于给定的路径上，但它应该确保路径有效。 例如，文件系统加载器确保路径位于有效的模板目录下。

**get_contents(origin)**

返回给定Origin实例的模板的内容。

这是文件系统加载器从文件系统读取内容的地方，或者数据库加载器将从数据库中读取的地方。 如果匹配的模板不存在，则会引发TemplateDoesNotExist错误。

**get_template(template_name, skip=None)[source]**

通过循环get_template_sources（）中的结果并调用get_contents（）来返回给定template_name的Template对象。 这将返回第一个匹配的模板。 如果未找到模板，则引发TemplateDoesNotExist。

可选的skip参数是扩展模板时要忽略的原点列表。 这允许模板扩展其他相同名称的模板。 它也用于避免递归错误。

通常，定义自定义模板加载器的get_template_sources（）和get_contents（）就足够了。 get_template（）通常不需要被覆盖。

# 模板原点

模板具有包含属性的来源，取决于它们从中加载的来源。

**`class Origin(name, template_name=None, loader=None)[source]`**

**name**

模板加载器返回的模板路径。 对于从文件系统读取的加载程序，这是模板的完整路径。

如果模板直接实例化而不是通过模板加载器实例化，则这是一个字符串值<unknown_source>。

**template_name**

传递给模板加载器的模板的相对路径。

如果模板直接实例化而不是通过模板加载器实例化，则这是无。

**loader**

构建此Origin的模板加载器实例。

如果模板直接实例化而不是通过模板加载器实例化，则这是无。

django.template.loaders.cached.Loader需要它的所有包装加载器来设置这个属性，通常通过用loader = self实例化Origin来实现。